# Abstract Instruments and paramters

Abstract parameters allow us to create abstract instrument types which are guarenteed to have certain parameters present. For instance, this will allow us to create a unified interface for all voltage sources.

Note: An instrument which contains abstract parameters shall be called an 'Abstract Instrument'

In [1]:
from qcodes import Instrument 

In [2]:
class BaseVoltageSource(Instrument): 
    """
    All abstract parameters *must* be implemented 
    before this class can be initialized. This 
    allows us to enforce an interface. 
    """
    
    def __init__(self, name: str): 
        super().__init__(name)
        
        self.add_parameter(
            "voltage", 
            unit="V", 
            abstract=True
        )
        
        self.add_parameter(
            "current", 
            unit="A", 
            get_cmd=None, 
            set_cmd=None 
        )

### We cannot instantiate an instrument with abstract parameters

In [4]:
try: 
    BaseVoltageSource("name")
except NotImplementedError as error:
    print(f"Error: {error}")

Error: Class 'BaseVoltageSource' has un-implemented Abstract Parameter(s): 'voltage'


Instruments which fail to initialize are not registered

In [16]:
BaseVoltageSource.instances()

[]

### Subclasses of Abstract instruments *must* implement all abstract parameters 

The following is correct 

In [8]:
class MyVoltageSource(BaseVoltageSource): 
    """
    A voltage source driver for a particular instrument 
    make and model. 
    """
    
    def __init__(self, name: str): 
        super().__init__(name)
        
        self.add_parameter(
            "voltage", 
            unit="V", 
            set_cmd=None, 
            get_cmd=None
        )

In [9]:
# We can use this voltage source like any other QCoDeS driver 
source = MyVoltageSource("name1")
source.voltage(0.1)
print(source.voltage())

0.1


In [15]:
MyVoltageSource.instances()

[<MyVoltageSource: name1>]

The following is wrong

In [18]:
class WrongSource(BaseVoltageSource): 
    """
    Let's 'forget' to implement the voltage parameter
    """

In [19]:
try: 
    WrongSource("name2")  
except NotImplementedError as error:
    print(f"Error: {error}")

Error: Class 'WrongSource' has un-implemented Abstract Parameter(s): 'voltage'


### Units of parameters defined in sub classes *must* match units defined in the base class 

In [20]:
class WrongSource2(BaseVoltageSource): 
    """
    We implement the voltage paramter with the wrong unit 
    """
    
    def __init__(self, name: str): 
        super().__init__(name)
        
        self.add_parameter(
           "voltage", 
           unit="mV"
        )   

In [21]:
try:
    WrongSource2("name4")  
except ValueError as error: 
    print(f"Error: {error}")

Error: The unit of the parameter 'voltage' is 'mV'. This is inconsistent with the unit defined in the base class


## How does this work?

We implement a `__post_init__` method on the `qcodes.Instrument` base class. The `__post_init__` will 

1. Check if all abstract parameters are implemented 
2. Register the instrument so it can be found with `qcodes.Instrument.find_instrument`

We illustrate this mechanism in the cells below

### `__post_init__`

In [34]:
class SomeBaseClass(Instrument): 
    
    def __init__(self, name):
        super().__init__(name)
        print("Initialzing Base")
    
    def __post_init__(self, name): 
        """
        This method is always called after the initialization of the sub class
        """
        super().__post_init__(name)  
        print("This is __post_init__")

In [35]:
class SubClass(SomeBaseClass): 
    
    def __init__(self, name): 
        super().__init__(name)
        print(f"Initializing Sub")

In [36]:
class SubSubClass(SubClass): 
    
    def __init__(self, name): 
        super().__init__(name)
        print(f"Initializing Sub Sub")

In [37]:
SubSubClass("foo3")

Initialzing Base
Initializing Sub
Initializing Sub Sub
This is __post_init__


<SubSubClass: foo3>

### I still don't understand how this works!

To understand how and when `__post_init__` is called, we must know that the regular `__init__` function of an `Instrument` subclass is monkey patched in the `__init_subclass__` method of the base class. In simplified code the following happens: 

```python

def __init_subclass__(cls): 
    original_init = cls.__init__
    def __new_init__(self, *args, **kwargs): 
        original_init(self, *args **kwargs)
        self.__post_init__(self, *args, **kwargs)
    
    # Monkey patch the init method of the sub class 
    cls.__init__ = __new_init__
```

We can see this by inspecting the `__init__` method of the subclass:

In [39]:
SubSubClass.__init__

<function qcodes.instrument.base.Instrument.__init_subclass__.<locals>.__new_init__(self, *args, **sub_class_kwargs)>

In practice the above code is a little more complex because we need to take care of higher order subclasses (e.g. sub-sub classes). To understand this, consider the init of a sub-sub class. In effect we are patching this method twice!

`Patch(Patch(__init__))`

If the above naive code is implemented, each `Patch` call would call the `__post_init__` method. To prevent this there is a `__init_call_count` class attribute which is initialized at zero. Before calling `original_init` the counter is incremented and after the call it is decremented. The `__post_init__` method is only called if `__init_call_count == 0`